In [1]:
# Imports
import csv
import json
import pathlib
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
from arcgis.gis import GIS

In [2]:
# Tests

test = 'WMS Get Maps'
timestamp = time.strftime("%Y%m%d-%H%M%S")

base_url = 'https://gis.earthdata.nasa.gov/image/rest/services/POWER/POWER_901_ANNUAL_METEOROLOGY_UTC/ImageServer'
wms_url = base_url + '/WMSServer?'
multi_url = base_url + '/multidimensionalInfo?f=pjson'

multi_get = requests.get(multi_url)
multi_content = multi_get.json()
variables = multi_content['multidimensionalInfo']['variables']
names = [var.get('name') for var in variables]

bbox = [-90, -180, 90, 180]
bbox_str = ','.join([str(x) for x in bbox])

start_time = time.time()
    
'''
    TODO:
        - Figure out which of these can be:
            - hardcoded
            - derived
            - function parameters
        - Other parameters are optional
'''
for var in names:
    # TODO: GEORGE can you check:
        # my adaptation of get_multiInfo code to get the time slices from the variables
        # my inclusion of first time extent into the optional parameters   
    if var.get("dimensions"):
        # Search for the time dimension and extract the first time extent value
        for dimension in var.get("dimensions"):
            if dimension.get("name") == "StdTime" or dimension.get("field") == "StdTime":
                time_extent = dimension.get("extent", [])
                if time_extent:
                    first_time_extent_value = time_extent[0]
                    last_time_extent_value = time_extent[-1]
                    break
    else:
        print("No dimensions found for the first variable.")
    
    # Calculate the time range
    time_range = None
    if first_time_extent_value:
        time_range = (first_time_extent_value, last_time_extent_value)
    
    params = {
        'request': 'GetMap',
        'service': 'WMS',
        'version': '1.1.1', # Service version. Value is one of 1.0.0, 1.1.0, 1.1.1, 1.3.0
        'layers': var, # Layers to display on map. Value is a comma-separated list of layer names.
        'styles': '', # Styles in which layers are to be rendered.
                      # Value is a comma-separated list of style names, or empty if default styling is required. 
                      # Style names may be empty in the list, to use default layer styling.
        'srs': 'EPSG:4326', # Spatial Reference System for map output.
                            # Value is in form EPSG:nnnn. crs is the parameter key used in WMS 1.3.0.
        'bbox': bbox_str, # Bounding box for map extent. Value is minx,miny,maxx,maxy in units of the SRS.
        'width': '780', # Width of map output, in pixels.
        'height': '330', # Height of map output, in pixels.
        'format': 'image/png' # Output format of map. Valid values are image/jpeg, image/png, and image/gif.
    }
       
    opt_params = {
        'transparent': 'False', # Whether the map background should be transparent. Values are true or false. Default is false
        'bgcolor': 'FFFFFF', # Background color for the map image. Value is in the form RRGGBB. Default is FFFFFF (white)
        'exceptions': '', # Format in which to report exceptions. Default value is application/vnd.ogc.se_xml
        'time': first_time_extent_value, # Time value or range for map data. See Time Support in GeoServer WMS for more information
                  # https://docs.geoserver.org/2.22.x/en/user/services/wms/time.html#wms-time
        'sld': '', # A URL referencing a StyledLayerDescriptor XML file which controls or enhances map layers and styling
        'sld_body': '' # A URL-encoded StyledLayerDescriptor XML document which controls or enhances map layers and styling
    }
    response = requests.get(wms_url, params=params)
    image_url = response.url

    export_path = pathlib.Path(f"exports/{test}_{var}_{timestamp}.png")
    with export_path.open("wb") as f:
        f.write(response.content)

